In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

import numpy as np
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [2]:
def load_data():
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Lambda(lambda x: x.view(-1))])

    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

    x_train = train_dataset.data.numpy().reshape(-1, 784) / 255.0
    x_test = test_dataset.data.numpy().reshape(-1, 784) / 255.0
    
    return x_train, x_test

In [15]:
def relu(x):
    """Applies the ReLU activation function element-wise.

    Parameters:
        x (np.ndarray): Input array.

    Returns:
        np.ndarray: Array with ReLU applied, where each element is the maximum 
                    of 0 and the input element.
    """
    return np.maximum(0, x)

def sigmoid(x):
    """Applies the Sigmoid activation function element-wise.

    Parameters:
        x (np.ndarray): Input array.

    Returns:
        np.ndarray: Array with Sigmoid applied to each element.
    """
    return 1 / (1 + np.exp(-x))

def lin_combination(x, weights, bias):
    """Computes a linear combination of inputs, weights, and bias.

    Parameters:
        x (np.ndarray): Input array.
        weights (np.ndarray): Weight matrix.
        bias (np.ndarray): Bias vector.

    Returns:
        np.ndarray: Result of the linear combination.
    """
    return np.dot(x, weights) + bias

def mse_loss(x, x_reconstructed):
    return np.mean((x - x_reconstructed)**2)

In [ ]:
class AutoEncoder:
    def __init__(self, epochs=100, lr=0.001):
        # Initialize weights and biases
        
        self.w1 = np.random.randn(784, 512) * 0.01
        self.b1 = np.zeros(512)
        
        self.w2 = np.random.randn(512, 256) * 0.01 
        self.b2 = np.zeros(256) 
        
        self.w3 = np.random.randn(256, 128) * 0.01
        self.b3 = np.zeros(128) 
        
        self.w4 = np.random.randn(128, 256) * 0.01
        self.b4 = np.zeros(256)
        
        self.w5 = np.random.randn(256, 512) * 0.01
        self.b5 = np.zeros(512)
        
        self.w6 = np.random.randn(512, 784) * 0.01
        self.b6 = np.zeros(784)

    def forward(self, X): 
        # Encoder: multiple layers with ReLU activation
        self.layer_1 = relu(lin_combination(x, w1, b1))
        self.layer_2 = relu(lin_combination(self.layer_1, w2, b2))
        self.layer_3 = relu(lin_combination(self.layer_2, w3, b3)) # Encoded
    
        # Decoder: multiple layers with ReLU activation for reconstruction
        self.layer_4 = relu(lin_combination(self.layer_3, w4, b4)) 
        self.layer_5 = relu(lin_combination(self.layer_4, w5, b5))
        self.layer_6 = relu(lin_combination(self.layer_5, w6, b6)) # Decoded
        
        return self.layer_3.T, self.layer_6.T

    def backward(self):
        pass

    def train(self, X, epochs, batch_size, lr):
        # Initialize lists for tracking metrics
        losses = []
        latent_codes = []
        reconstructed_data = []
        
        start_time = time.time()  # Start measuring time
    
        num_samples = X.shape[0]  # Define number of samples in dataset
    
        for epoch in range(epochs):
            # Shuffle data at the start of each epoch
            indices = np.random.permutation(num_samples)
            X_shuffled = X[indices]
            epoch_loss = 0
    
            # Divide data into mini-batches
            for i in range(0, num_samples, batch_size):
                X_batch = X_shuffled[i:i + batch_size] #  Extract the mini-batch
                output = self.forward(X_batch) #  Forward pass to get reconstructed output

                # Calculate loss (mean squared error between input and reconstruction)
                loss = mse_loss(X_batch, output)
                epoch_loss += loss 
                
                # Backpropagation
                self.backward(X_batch, X_batch, output, lr)
    
            # Calculate and store the average loss for this epoch
            avg_loss = epoch_loss / (num_samples / batch_size)
            losses.append(avg_loss)
            
            # Optional: Collect latent codes and reconstructed data at intervals (e.g., every 10 epochs)
            if epoch % 10 == 0:
                latent_codes.append(self.get_latent(X))  # Placeholder for function to get latent codes
                reconstructed_data.append(self.forward(X))  # Forward pass on full dataset for reconstruction
    
            print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
        
        print("Training complete in {:.2f} seconds.".format(time.time() - start_time))

    return losses, latent_codes, reconstructed_data
        

In [ ]:
# 8. Plot the loss curve
def plot_loss_curve(losses):
    plt.figure(figsize=(8, 6))
    plt.plot(losses, label='MSE Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Curve')
    plt.legend()
    plt.show()

# Uncomment after training to plot the loss curve
# plot_loss_curve(losses)

# 9. Visualize latent codes using PCA
def visualize_latent_space(latent_codes):
    pca = PCA(n_components=2)
    reduced_codes = pca.fit_transform(latent_codes)
    plt.figure(figsize=(8, 6))
    plt.scatter(reduced_codes[:, 0], reduced_codes[:, 1], s=5, alpha=0.6)
    plt.title('Latent Space Visualization (PCA)')
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.show()

# Uncomment after training to visualize latent space
# visualize_latent_space(latent_codes)

# 10. Show original and reconstructed images side by side
def show_images(original, reconstructed, n=10):
    plt.figure(figsize=(20, 4))
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(original[i].reshape(28, 28), cmap='gray')
        plt.axis('off')

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(reconstructed[i].reshape(28, 28), cmap='gray')
        plt.axis('off')
    plt.show()

# Uncomment after training to show images
# show_images(x_train, reconstructed_data)